In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
print("Imported Successfully")

# Load the image data
train_images = []
train_labels = []
input_dir = '/kaggle/input/test-train-data/test_train_data'
for image_file in os.listdir(input_dir):
    image = cv2.imread(os.path.join(input_dir, image_file))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (64, 64))
    image = np.stack([image, image, image], axis=2)
    train_images.append(image)
    train_labels.append(int(image_file.split('_')[1].split('.')[0]))
print("Loaded Successfully")

# Preprocess the image data
train_images = np.array(train_images, dtype=object)
train_images = train_images / 255.0
train_labels = np.array(train_labels)
train_images = np.array(train_images, dtype=np.float32)
val_split = 0.2
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=val_split, random_state=42)
num_classes = 60000
print("Success")

# Build the CNN model
model = Sequential()
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
model.add(resnet50)
model.add(Flatten())
model.add(Dense(num_classes, activation='sigmoid'))
print(model, "\nModel Built")

# Define a custom loss function to handle labels outside the valid range
# def custom_loss(y_true, y_pred):
#     invalid_labels = K.cast(K.greater_equal(y_true, 49), dtype=K.floatx()
#     loss = K.sparse_categorical_crossentropy(y_true, y_pred) * (1 - invalid_labels)
#     return loss

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model, "\nModel Compiled")

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=100, validation_data=(val_images, val_labels))
model.save('trained_model_10k.h5')